In [369]:
#importing Packages and framework
import pandas as pd
import numpy as np
#Import ast 
import ast

from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from nltk.stem.porter import PorterStemmer

#similarities between two vectors
from sklearn.metrics.pairwise import cosine_similarity

In [370]:
#Importing data
movie_credits = pd.read_csv('datasets/credits.csv')
movie_info = pd.read_csv('datasets/movies.csv')

In [371]:
# Checking the data
movie_credits.head(3)
movie_info.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [372]:
#Merging the two dataframe
movies = movie_credits.merge(movie_info, on= 'title')

In [373]:
#Checking for missing values
movies.isna().sum()

movie_id                   0
title                      0
cast                       0
crew                       0
budget                     0
genres                     0
homepage                3095
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
vote_average               0
vote_count                 0
dtype: int64

In [374]:
#Selecting needed columns
selected_movies_columns = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [375]:
# Checking the selected data
selected_movies_columns.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [376]:
# Checking for missing value
selected_movies_columns.isna().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [377]:
# Dropping missing values
selected_movies_columns.dropna(inplace=True)

<ipython-input-377-ce59b2c5b6a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns.dropna(inplace=True)


In [378]:
# Checking for duplicate values
selected_movies_columns.duplicated().sum()

0

In [379]:
#Function for converting data to real strings for keywords and genres
def convert_keywords_genres(obj):
    link = []
    for i in ast.literal_eval(obj):
        link.append(i['name'])
    return link

In [380]:
#Converting keywords and genres data to real strings
selected_movies_columns['keywords'] = selected_movies_columns['keywords'].apply(convert_keywords_genres)
selected_movies_columns['genres'] = selected_movies_columns['genres'].apply(convert_keywords_genres)


<ipython-input-380-4bb6e54f2e3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['keywords'] = selected_movies_columns['keywords'].apply(convert_keywords_genres)
<ipython-input-380-4bb6e54f2e3f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['genres'] = selected_movies_columns['genres'].apply(convert_keywords_genres)


In [381]:
#Checking converted data
selected_movies_columns.head(4)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."


In [382]:
#Function for converting data to real strings for cast
def convert_cast(obj):
    link2 = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            link2.append(i['name'])
            counter += 1
        else:
            break
        return link2    


In [383]:
#Converting cast data to real strings
selected_movies_columns['cast'] = selected_movies_columns['cast'].apply(convert_cast)

<ipython-input-383-0c815e0c6808>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['cast'] = selected_movies_columns['cast'].apply(convert_cast)


In [384]:
# Check converted cast string
selected_movies_columns.head(4)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",[Sam Worthington],"[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",[Johnny Depp],"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[Daniel Craig],"[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",[Christian Bale],"[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."


In [385]:
#Function for converting data to real strings for directors
def get_director(obj):
    link3 = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
          link3.append(i['name'])
          break
    return link3    

In [386]:
#Converting director data to real strings
selected_movies_columns['crew']=selected_movies_columns['crew'].apply(get_director)

<ipython-input-386-30b5f31fda80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['crew']=selected_movies_columns['crew'].apply(get_director)


In [387]:
# Check converted director string
selected_movies_columns.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",[Sam Worthington],[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",[Johnny Depp],[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[Daniel Craig],[Sam Mendes]


In [388]:
#Splitting long overview sentences into words
selected_movies_columns['overview'] = selected_movies_columns['overview'].apply(lambda x:x.split())


<ipython-input-388-3c1433def809>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['overview'] = selected_movies_columns['overview'].apply(lambda x:x.split())


In [389]:
#Closing up spaces between words in genres, keywords, cast and crew
selected_movies_columns['genres'] = selected_movies_columns['genres'].apply(lambda x:[i.replace(" ","") for i in x])
selected_movies_columns['keywords'] = selected_movies_columns['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
#selected_movies_columns['cast'] = selected_movies_columns['cast'].apply(lambda x:[i.replace(" ","") for i in x])
selected_movies_columns['crew'] = selected_movies_columns['crew'].apply(lambda x:[i.replace(" ","") for i in x])







<ipython-input-389-08079db6c214>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['genres'] = selected_movies_columns['genres'].apply(lambda x:[i.replace(" ","") for i in x])
<ipython-input-389-08079db6c214>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['keywords'] = selected_movies_columns['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
<ipython-input-389-08079db6c214>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [390]:
#Checking closed words
selected_movies_columns.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",[Sam Worthington],[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",[Johnny Depp],[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",[Daniel Craig],[SamMendes]


In [391]:
# Creating tag column by adding overview,genres, keywords and crew together.
selected_movies_columns['tags']= selected_movies_columns['overview']+selected_movies_columns['genres']+selected_movies_columns['keywords']+selected_movies_columns['crew']

<ipython-input-391-fba102835c0b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_movies_columns['tags']= selected_movies_columns['overview']+selected_movies_columns['genres']+selected_movies_columns['keywords']+selected_movies_columns['crew']


In [392]:
# Checking created tags
selected_movies_columns.head(3)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",[Sam Worthington],[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",[Johnny Depp],[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",[Daniel Craig],[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."


In [393]:
#Creating new dataframe by filtering movie id, title and tags only
main_df = selected_movies_columns[['movie_id', 'title', 'tags']]

In [394]:
#Joining splitted words in tags
main_df['tags']= main_df['tags'].apply(lambda x:' '.join(x))

<ipython-input-394-5adb12972927>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['tags']= main_df['tags'].apply(lambda x:' '.join(x))


In [395]:
#Checking the joined tags
main_df.head(3)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...


In [396]:
#Converting the tags to lower case to increase performance
main_df['tags'] = main_df['tags'].apply(lambda x: x.lower())

<ipython-input-396-a3575a4081c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['tags'] = main_df['tags'].apply(lambda x: x.lower())


In [397]:
#Making sure the lower case is affective
main_df.head(4)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...


In [398]:
#Initializing CountVectorizer object

cv = CountVectorizer(max_features=500,stop_words='english')

In [399]:
#Converting words to vectors and checking the dimensions
cv.fit_transform(main_df['tags']).toarray().shape

(4805, 500)

In [400]:
#Converting words to vectors and saving them.
vectors = cv.fit_transform(main_df['tags']).toarray()

In [401]:
#Checking the vectors
vectors[0]

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [402]:
#Checking for the length of vector feature names
len(cv.get_feature_names())

500

In [403]:
#Initializing porterstemmer object. This is used to remove suffixes from English words and extract their stems (term normalization)
ps = PorterStemmer()

In [404]:
#Function for term normalization
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [405]:
#Extracting the stem of the tags
main_df['tags']= main_df['tags'].apply(stem)

<ipython-input-405-2c5933bda0e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['tags']= main_df['tags'].apply(stem)


In [406]:
#Measuring similarity between two vectors 
cosine_similarity(vectors)

array([[1.        , 0.21938173, 0.12524486, ..., 0.05292561, 0.05564149,
        0.        ],
       [0.21938173, 1.        , 0.15569979, ..., 0.06579517, 0.        ,
        0.        ],
       [0.12524486, 0.15569979, 1.        , ..., 0.05634362, 0.        ,
        0.        ],
       ...,
       [0.05292561, 0.06579517, 0.05634362, ..., 1.        , 0.15018785,
        0.06579517],
       [0.05564149, 0.        , 0.        , ..., 0.15018785, 1.        ,
        0.13834289],
       [0.        , 0.        , 0.        , ..., 0.06579517, 0.13834289,
        1.        ]])

In [407]:
#Checking the dimensions of the similariy vectors
cosine_similarity(vectors).shape

(4805, 4805)

In [408]:
#Saving extracted similarities
similarity = cosine_similarity(vectors)

In [409]:
#Viewing similarities of the first index of the vectors
similarity[0]

array([1.        , 0.21938173, 0.12524486, ..., 0.05292561, 0.05564149,
       0.        ])

In [410]:
#Viewing the shape of the similarities of the first index of the vectors
similarity[0].shape

(4805,)

In [411]:
#Sorting similarities of the first index of the vectors in descending order
sorted(list(enumerate(similarity[0])), reverse=True, key =lambda x:x[1])[1:6]

[(507, 0.5821817364274594),
 (151, 0.4950737714883372),
 (322, 0.4900980294098035),
 (1194, 0.4900980294098035),
 (1216, 0.48507125007266594)]

In [412]:
#Movie recommedation function
def recommend(movie):
    movie_index=main_df[main_df['title']==movie].index[0]
    distance = similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)), reverse=True, key= lambda x:x[1])[1:6]

    for i in movies_list:
        print(main_df.iloc[i[0]].title)

In [416]:
#Calling the recommendation function to predict movies
recommend('Identity')

Switchback
Alex Cross
The Raven
Eye of the Beholder
Sorority Row
